# Laboratorio #2 – Detección de SPAM
### Jose Hernandez 20053
### Javier Mombiela 20067

In [1]:
# importando librerias
import re
import nltk
import pandas as pd
import contractions
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

C:\Users\rjmom\AppData\Local\Temp\ipykernel_29684\123822451.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Parte 1 – Ingeniería de características 
### 1.1 - Exploración de datos y Pre-procesamiento

In [2]:
# Cargando datos y eliminando columnas vacias
data = pd.read_csv("spam.csv", encoding='latin-1')
data = data.iloc[:, :2]

data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Aplicando tecnicas de pre-procesamiento de lenguaje natural

In [3]:
# Convertir texto a minúsculas en la columna 'v2'
data['v2'] = data['v2'].str.lower()

data.head()

,v1,v2
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."


In [4]:
# Expandir contracciones
print("Original  : ", data['v2'].iloc[4])
data['v2'] = data['v2'].apply(lambda x: contractions.fix(x))

# data.head()
print("Modificado: ", data['v2'].iloc[4])

Original  :  nah i don't think he goes to usf, he lives around here though
Modificado:  nah i do not think he goes to usf, he lives around here though


In [5]:
# Remover caracteres especiales, números y símbolos
print("Original  : ", data['v2'].iloc[12])
data['v2'] = data['v2'].apply(lambda x: re.sub(r'[^a-z\s]', '', x))

print("Modificado: ", data['v2'].iloc[12])

Original  :  urgent! you have won a 1 week free membership in our å£100,000 prize jackpot! txt the word: claim to no: 81010 t&c www.dbuk.net lccltd pobox 4403ldnw1a7rw18
Modificado:  urgent you have won a  week free membership in our  prize jackpot txt the word claim to no  tc wwwdbuknet lccltd pobox ldnwarw


In [6]:
# Eliminando stopwords
nltk.download('stopwords')
print("Original  : ", data['v2'].iloc[77])

stop_words = set(stopwords.words('english'))
data['v2'] = data['v2'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

print("Modificado: ", data['v2'].iloc[77])

Original  :  i like you peoples very much but am very shy pa
Modificado:  like peoples much shy pa


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rjmom\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# Stemming
porter_stemmer = PorterStemmer()
print("Original  : ", data['v2'].iloc[13])

def stem_text(text):
    return ' '.join([porter_stemmer.stem(word) for word in text.split()])

data['v2'] = data['v2'].apply(stem_text)
    
print("Modificado: ", data['v2'].iloc[13])

Original  :  searching right words thank breather promise take help granted fulfil promise wonderful blessing times
Modificado:  search right word thank breather promis take help grant fulfil promis wonder bless time


Representacion de Texto